<a href="https://colab.research.google.com/github/AlexUmnov/genai_course/blob/main/week2_llm_agents/bonus_chain_to_lcel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The world of LLM and LLM related tools changes very fast.

For example, while we were writing this course, one of most widely used libraries for LLM-related tasks deprecated their main interface.

More precisely they went from `Chain` interface to `LCEL` - Lang Chain Expression Language.

This change does not change the fundamental materials so much. But we wanted to show you the difference between the two, as you might see tutorials in LCEL and we want you to be prepared

In [5]:
!pip install langchain openai langchain_openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [6]:
import os
os.environ['OPENAI_API_KEY'] = open("../../keys/.open-ai-api-key").read().strip()

A short demonstration of how the new interface looks:

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

print(chain.invoke({"topic": "beavers and elves"}))

Why don't elves play hide and seek with beavers?

Because they always find them "barking" up the wrong tree!


This notebook will have a demonstration of all the concepts introduced in the practice sesion and how (or if) they changed with the new version.

# Using a model

Before we were using a method `predict` and now the correct one is `invoke`

## Classic LLM

In [11]:
from langchain.llms import OpenAI

llm = OpenAI()

llm.invoke("Hello ")

'\n\nHello! How are you doing today? '

In [14]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("One {object} a day keeps who away?")

prompt_value = prompt.invoke({"object": "apple"})

llm.invoke(prompt_value.to_string())

'\n\nDoctor.'

## Chat model

In [16]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

chat_model.invoke(prompt_value.to_messages())

AIMessage(content='The saying "One apple a day keeps the doctor away."')

# Chains

What it looked like before

```
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
chain.run("Australia"
```

Now the interface is the following

In [20]:
chain = prompt | model
chain.invoke({"object": "Banana"})

AIMessage(content='There isn\'t a specific person or thing that one banana a day keeps away. However, it is a healthy habit that may help to keep diseases away due to its high content of vitamins and minerals. The phrase is a play on the old saying "An apple a day keeps the doctor away."')

## Sequential chains

Before it looked like this:

```
from langchain.chains import SimpleSequentialChain

first_prompt = PromptTemplate.from_template(
    "What is the capital of {country}?"
)
first_chain = LLMChain(llm=llm, prompt=first_prompt)

second_prompt = PromptTemplate.from_template(
    "{city} is the capital of which country?"
)
second_chain = LLMChain(llm=llm, prompt=second_prompt)

simple_sequential_chain = SimpleSequentialChain(
    chains=[first_chain, second_chain],
    verbose=True
)
```
And the new interface is the following:


In [26]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

output_parser = StrOutputParser()

pie_prompt = ChatPromptTemplate.from_template("I want to bake a {pie}. Give me a list of ingredients.")

looking_in_the_fridge_prompt = ChatPromptTemplate.from_template(
    "I have {in_the_fridge}, repeat {recipe} adding to each ingredient if I need to buy it"
)

model = ChatOpenAI()

recipe_chain = pie_prompt | model | output_parser

ingredients_chain = (
   {"in_the_fridge": itemgetter("in_the_fridge"), "recipe": recipe_chain}
   | looking_in_the_fridge_prompt
   | model
   | output_parser
)

print(
    ingredients_chain.invoke(
        {"pie": "cheescake", "in_the_fridge": "milk"},
    )
)

If you need to buy any of the ingredients, here is the additional information you may need:

- Graham cracker crumbs or digestive biscuits: You will need 1 ½ cups. These can usually be found in the baking aisle of most grocery stores.

- Granulated sugar: You will need a total of 1 ¼ cups (1/4 cup for the crust, 1 cup for the filling, and 2 tablespoons for the topping). This is a common ingredient that can be found in the baking aisle.

- Unsalted butter: You will need ½ cup for the crust. Check your pantry to see if you have enough, otherwise you can find unsalted butter in the dairy section of most grocery stores.

- Cream cheese: You will need 3 packages, which usually come in 8 oz each, totaling 24 oz. You can find cream cheese in the dairy section of most grocery stores.

- Vanilla extract: You will need 1 ½ teaspoons in total (1 teaspoon for the filling and ½ teaspoon for the topping). This can be found in the baking aisle of most grocery stores.

- Eggs: You will need 3 large eg

## Debugging

In [36]:
from langchain.callbacks.tracers import ConsoleCallbackHandler

print(
    ingredients_chain.invoke(
        {"pie": "cheescake", "in_the_fridge": "milk"},
        config={'callbacks': [ConsoleCallbackHandler()]}
    )
)

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "pie": "cheescake",
  "in_the_fridge": "milk"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<in_the_fridge,recipe>] Entering Chain run with input:
{
  "pie": "cheescake",
  "in_the_fridge": "milk"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<in_the_fridge,recipe> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "pie": "cheescake",
  "in_the_fridge": "milk"
}[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<in_the_fridge,recipe> > 4:chain:RunnableLambda] Entering Chain run with input:
{
  "pie": "cheescake",
  "in_the_fridge": "milk"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<in_the_fridge,recipe> > 4:chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": "milk"
}

[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<in_the_fridge,recipe> > 3:chain:RunnableSequence > 4:prompt:ChatProm

# Using tools

Before it looked like this:

```
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI
from langchain.tools import DuckDuckGoSearchRun

llm = OpenAI(temperature=0)

tools = [DuckDuckGoSearchRun()]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
```

In [27]:
!pip install --upgrade --quiet  langchain langchain-openai duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 21.4 MB/s eta 0:00:00


In the new version it looks like this:


In [29]:
from langchain.tools import DuckDuckGoSearchRun
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

search = DuckDuckGoSearchRun()

template = """turn the following user input into a search query for a search engine:

{input}"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = prompt | model | StrOutputParser() | search

chain.invoke({"input": "What are the most famous K-pop bands?"})

'While famous groups like BTS and Blackpink have achieved global success, there are always rising stars set on becoming the most popular K-pop group ever. Vote up the top K-pop groups of 2021, and then check out how many of these underrated K-pop groups you know! Most divisive: EXO-CBX 1 BTS Big Hit Music (HYBE Labels) 95,793 votes Sports #Alicia Keys Griff COLLECTION46 LISTS The Best of K-pop The Best K-pop Boy Groups Of All Time Over 95.8K Ranker voters have come together to rank this list of The Best K-pop Boy Groups Of All Time Vote up your favorite Kpop boy bands and groups! 1. BTS BTS (방탄소년단) \'Butter\' Official MV There\'s no talking about K-pop without talking about BTS. Formed in 2013, the band—originally named the Bangtan Boys—has been going strong ever since. BTS is arguably the most popular K-pop group internationally, earning record-breaking album sales and even a Grammy nomination.BLACKPINK is one of the most successful K-pop girl groups, known for their catchy songs and 

# Using memory

Before

```
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain


prefix = """Have a conversation with a human, answering the following """\
    """questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

llm_chain = LLMChain(llm=OpenAI(), prompt=prompt)
agent = ZeroShotAgent(
    llm_chain=llm_chain,
    tools=tools,
    verbose=True
)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)
```

In [30]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)

chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | model
)

In [34]:
inputs = {"input": "I want to get a cat, do you think it's a good idea?"}
response = chain.invoke(inputs)
response

AIMessage(content="As an AI, I don't have personal opinions, but I can provide you with some information to help you make an informed decision. Getting a cat can be a wonderful experience for many people. Cats can provide companionship, reduce stress, and bring joy to your life. However, owning a cat also comes with responsibilities. You will need to provide proper care, including feeding, grooming, and regular veterinary visits. You should also consider if you have enough time and resources to devote to a cat's needs, including playtime and creating a safe environment for them. Additionally, if you have any allergies or live in a place with restrictions on pet ownership, those factors should be taken into account as well. Ultimately, it's important to consider your lifestyle, commitment, and ability to care for a cat before making a decision.")

In [37]:
memory.save_context(inputs, {"output": response.content})

In [38]:
inputs = {"input": "I want it, what would be a good name for it?"}
response = chain.invoke(inputs)
response

AIMessage(content='Choosing a name for your cat can be a fun and creative process! Here are a few suggestions to get you started:\n\n1. Whiskers\n2. Luna\n3. Oliver\n4. Bella\n5. Simba\n6. Nala\n7. Charlie\n8. Lucy\n9. Max\n10. Daisy\n\nRemember, the perfect name for your cat is personal and can reflect their personality, appearance, or any other qualities that you find endearing. Take your time to explore different options and choose a name that resonates with you and your new furry friend.')